# Convert SQLite outputs to parquet files with CytoTable

## Import libraries

In [1]:
import pathlib
import pandas as pd
import pprint
import os

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# Optional: set `PLATEMAP_LAYOUT` env var to process only a single platemap (e.g. 'platemap_1')
platemap_to_process = os.environ.get("PLATEMAP_LAYOUT")
# platemap_to_process = "platemap_1"  # for testing only

# set base directory for where the SQLite files are located (should be local to repo)
base_dir = pathlib.Path("../2.cellprofiler_processing/cp_output/").resolve(strict=True)

# Decide what to process
if platemap_to_process:
    print(f"Processing {platemap_to_process}")
    layouts = [platemap_to_process]
else:
    print("No specific layout set, processing all available platemaps")
    layouts = [p.name for p in base_dir.glob("platemap_*") if p.is_dir()]

pprint.pprint(layouts)

No specific layout set, processing all available platemaps
['platemap_3', 'platemap_4', 'platemap_1', 'platemap_2']


In [3]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Count_Cells,",
)

# type of file output from cytotable
dest_datatype = "parquet"

# set path to directory with SQLite files
sqlite_dir = pathlib.Path(f"{base_dir}/{platemap_to_process}").resolve(strict=True)

# directory for processed data
output_dir = pathlib.Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

for file_path in sqlite_dir.iterdir():
    plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 4 plates in this dataset. Below are the names:
CARD-CelIns-CX7_251125110001
CARD-CelIns-CX7_251023210001
CARD-CelIns-CX7_251126130001
CARD-CelIns-CX7_251124150001


## Convert SQLite to parquet files

In [4]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(
        f"{output_dir}/{platemap_to_process}/converted_profiles/{file_path.stem}_converted.parquet"
    )
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # Merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=5000,
    )

print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: CARD-CelIns-CX7_251125110001
Starting conversion with cytotable for plate: CARD-CelIns-CX7_251023210001
Starting conversion with cytotable for plate: CARD-CelIns-CX7_251126130001
Starting conversion with cytotable for plate: CARD-CelIns-CX7_251124150001
All plates have been converted with cytotable!


# Load in converted profiles to update

We will rename some of the columns (e.g., location centroids and cell count per FOV) to include Metadata prefix.

In [5]:
# Directory with converted profiles
converted_dir = pathlib.Path(
    f"{output_dir}/{platemap_to_process}/converted_profiles/"
).resolve(strict=True)

# List of columns to update with the "Metadata_" prefix
metadata_columns_to_update = [
    "Nuclei_Location_Center_X",
    "Nuclei_Location_Center_Y",
    "Cells_Location_Center_X",
    "Cells_Location_Center_Y",
    "Image_Count_Cells",
]

for file_path in converted_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)

    # If any, drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])

    # Rearrange columns and add "Metadata" prefix in one line
    df = df[
        metadata_columns_to_update
        + [col for col in df.columns if col not in metadata_columns_to_update]
    ].rename(
        columns=lambda col: (
            "Metadata_" + col if col in metadata_columns_to_update else col
        )
    )

    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)

## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [6]:
converted_df = pd.read_parquet(
    f"./data/{platemap_to_process}/converted_profiles/{plate_names[1]}_converted.parquet"
)

print(converted_df.shape)
converted_df.head()

(7387, 2315)


,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_ER_3_02_256,Nuclei_Texture_Variance_ER_3_03_256,Nuclei_Texture_Variance_Mito_3_00_256,Nuclei_Texture_Variance_Mito_3_01_256,Nuclei_Texture_Variance_Mito_3_02_256,Nuclei_Texture_Variance_Mito_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,458.578050,210.249489,473.362851,225.846736,5,7,CARD-CelIns-CX7_251023210001,f07,B02,1,...,9.726683,8.585116,0.731660,0.658796,0.701145,0.702682,2.573974,2.464123,2.471056,2.484499
1,1007.254802,25.928937,1007.243762,25.919386,10,8,CARD-CelIns-CX7_251023210001,f08,B02,1,...,10.162068,9.814562,15.584487,10.843501,14.687934,17.042039,4.920192,4.342821,4.538756,4.322995
2,550.477571,45.057971,550.464828,45.049655,18,16,CARD-CelIns-CX7_251023210001,f17,B02,1,...,20.771137,17.780316,1.374533,1.310761,1.641075,1.490930,3.971751,4.014045,5.169231,4.255472
3,731.717236,151.571937,735.436817,148.127582,5,29,CARD-CelIns-CX7_251023210001,f17,B03,1,...,2.799449,2.622891,1.580253,1.562574,1.575644,1.506316,2.984921,3.123117,3.101468,2.936457
4,342.702766,315.055327,330.828047,335.911988,4,37,CARD-CelIns-CX7_251023210001,f03,B04,1,...,15.617408,15.278007,55.448696,56.044557,54.999029,55.250192,19.121087,18.669289,19.121714,18.636818
